In [5]:
from paths import *
import pandas as pd
import numpy as np
from SM_tools import * 

In [6]:
#select the water year of interest 
water_year = 2019

#select the variable of interest
var = 'elev'

#date assim initiated - used for labeling output directory
today = datetime.date.today()
hoy = today.strftime("%Y-%m-%d")
    
# snotel swe timeseries 
snotel_swe_all = pd.read_csv(dataPath + 'SNOTEL_data_SWEDmeters'+ str(water_year -1) +'-09-01_'+str(water_year)+'-09-30.csv', index_col=0, parse_dates=True)

# swe timeseries of stations to be assimilated 
snotel_swe_assim = snotel_swe_all[np.intersect1d(snotel_swe_all.columns, snotel_assim_sites.code.values)]

# cso geodataframe 
cso_gdf = gpd.read_file(dataPath+'all_cso_'+str(water_year -1) +'_'+str(water_year)+'_meta.geojson')
cso_gdf['dt'] = pd.DatetimeIndex(cso_gdf['timestamp'])

#add line to edit par to reflect assim year -> edit met file name and start year in par file

In [ ]:
#dictionalry of par variables
with open('par_base.json') as f:
    base = json.load(f)
    
#Edit the par file to set parameters with new values
def edit_par(par_dict,parameter,new_value):
    lines = open(parFile, 'r').readlines()
    if par_dict[parameter][2] == 14 or par_dict[parameter][2] == 17 \
    or par_dict[parameter][2] == 18 or par_dict[parameter][2] == 19 \
    or par_dict[parameter][2] == 93 or par_dict[parameter][2] == 95 \
    or par_dict[parameter][2] == 97 or par_dict[parameter][2] == 100 \
    or par_dict[parameter][2] == 102 or par_dict[parameter][2] == 104 \
    or par_dict[parameter][2] == 107 or par_dict[parameter][2] == 108:
        text = str(new_value)+'\n'
    else:
        text = str(new_value)+'\t\t\t!'+par_dict[parameter][1]
    lines[par_dict[parameter][2]] = text
    out = open(parFile, 'w')
    out.writelines(lines)
    out.close()

In [ ]:
edit_par(base,'iyear_init',str(water_year))
edit_par(base,'met_input_fname','met/mm_WY_'+str(water_year -1)+'-'+str(water_year)+'.dat')

## run SM for CSO

In [6]:
!pwd

/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev


In [3]:
out_path = gdat_out_path+str(water_year)+'assim_run'+hoy+'/'

In [4]:
#assimilate CSO observations into SM
SMassim_ensemble(cso_gdf,var,out_path)


edges: [1774.  2062.4 2350.8 2639.2 2927.6 3216. ]
labels: [0 1 2 3 4]
/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev/code


/nfs/attic/dfh/miniconda/envs/snowmodelcal/lib/python3.6/site-packages/geopandas/geodataframe.py:831: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev

 For a DATA ASSIMILATION RUN, MAX_OBS_DATES must be
 defined in SNOWMODEL.INC to be greater than the
 number of observation dates in the entire simulation
 + (plus) the number of years in the simulation.  For
 example, for a 6-year simulation with two observation
 dates in each year, you would set max_obs_dates to be
 at least = 18.

^C
mv: cannot stat ‘/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev/outputs/wi_assim/swed.gdat’: No such file or directory
/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev/code
/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev

 For a DATA ASSIMILATION RUN, MAX_OBS_DATES must be
 defined in SNOWMODEL.INC to be greater than the
 number of observation dates in the entire simulation
 + (plus) the number of years in the simulation.  For
 example, for a 6-year simulation with two observation
 dates in each year, you would set max_obs_dates to be
 at lea

## run SM for SNOTEL

In [5]:
#assimilate SNOTEL observations into SM
SMassim_ensemble_snotel(cso_gdf,snotel_assim_sites,snotel_swe_assim,var,out_path)


edges: [1774.  2062.4 2350.8 2639.2 2927.6 3216. ]
labels: [0 1 2 3 4]
/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev/code
/nfs/attic/dfh/Aragon2/WY_scratch/jan2021_snowmodel-dfhill_elev

 For a DATA ASSIMILATION RUN, MAX_OBS_DATES must be
 defined in SNOWMODEL.INC to be greater than the
 number of observation dates in the entire simulation
 + (plus) the number of years in the simulation.  For
 example, for a 6-year simulation with two observation
 dates in each year, you would set max_obs_dates to be
 at least = 18.


 Checking for sufficient met forcing data to
   complete the model simulation.  This may
   take a while, depending on how big your met
   input file is.


 You are running the large-domain Barnes oi scheme
   This requires:
   1) no missing data for the fields of interest
   2) no missing stations during the simulation
   3) met file must list stations in the same order
   4) the number of nearest stations used is 9 or less
   5)  **** no error checking

## run SM for CSO & SNOTEL

In [21]:
#assimilate SNOTEL observations into SM
SMassim_ensemble_both(snotel_swe_assim,snotel_assim_sites,cso_gdf,var,out_path)

In [8]:
STswe = snotel_swe_assim
STmeta = snotel_assim_sites
CSOdata = cso_gdf
outFpath = 'test.dat'

In [22]:
var == 'elev'
edges = np.histogram_bin_edges(CSOdata.dem_elev,bins=5, range=(CSOdata.dem_elev.min(),CSOdata.dem_elev.max()))
print('edges:',edges)
labs = np.arange(0,len(edges)-1,1)
print('labels:',labs)
bins = pd.cut(STmeta['dem_elev'], edges,labels=labs)
STmeta['elev_bin']=bins 
bins = pd.cut(CSOdata['dem_elev'], edges,labels=labs)
CSOdata['elev_bin']=bins 
for lab in labs:
    newST = STmeta[STmeta.elev_bin == lab]
    newCSO = CSOdata[CSOdata.elev_bin == lab]
    newSTswe = STswe[np.intersect1d(STswe.columns, newST.code.values)] 
    outFpath = 'test_'+str(lab)+'.dat'
    num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
num_obs

edges: [1774.  2062.4 2350.8 2639.2 2927.6 3216. ]
labels: [0 1 2 3 4]


395

In [32]:
var = 'M'
newST = STmeta
mo = [11,12,1,2,3,4,5]#np.unique(STswe.index.month)
for m in mo:
    newSTswe = STswe[STswe.index.month == m]
    newCSO = CSOdata[CSOdata[var] == m]       
    outFpath = 'test_'+str(m)+'.dat'
    num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
num_obs

31

In [28]:
def make_SMassim_file_both(STswe,STmeta,CSOdata,outFpath):

    f= open(outFpath,"w+")
    
    #determine number of days with observations to assimilate
    if STswe.shape[1]>0:
        uq_day = np.unique(np.concatenate((STswe.index.date,CSOdata.dt.dt.date.values)))
        f.write('{:02.0f}\n'.format(len(uq_day)))
    else:
        uq_day = np.unique(CSOdata.dt.dt.date.values)
        f.write('{:02.0f}\n'.format(len(uq_day)))
    
    # determine snotel stations 
    stn = list(STswe.columns)
    
    # ids for CSO observations - outside of loop so each observation is unique
    IDS = 500
    
    #add assimilation observations to output file
    for i in range(len(uq_day)):

        SThoy = STswe[STswe.index.date == uq_day[i]]
        CSOhoy = CSOdata[CSOdata.dt.dt.date.values == uq_day[i]]

        d=uq_day[i].day
        m=uq_day[i].month
        y=uq_day[i].year

        date = str(y)+' '+str(m)+' '+str(d)

        stn_count = len(stn) + len(CSOhoy)
        
        if stn_count > 0:
            f.write(date+' \n')
            f.write(str(stn_count)+' \n')

        #go through snotel stations for that day 
        ids = 100
        if len(SThoy) > 0:
            for k in stn:
                ids = ids + 1 
                x = STmeta.easting.values[STmeta.code.values == k][0]
                y = STmeta.northing.values[STmeta.code.values == k][0]
                swe = SThoy[k].values[0]
                f.write('{:3.0f}\t'.format(ids)+'{:10.0f}\t'.format(x)+'{:10.0f}\t'.format(y)+'{:3.2f}\n'.format(swe))    
        #go through cso obs for that day 
        if len(CSOhoy) > 0:
            for c in range(len(CSOhoy)):
                IDS = IDS + 1 
                x= CSOhoy.x[CSOhoy.index[c]]
                y=CSOhoy.y[CSOhoy.index[c]]
                swe=CSOhoy.swe[CSOhoy.index[c]]
                f.write('{:3.0f}\t'.format(IDS)+'{:10.0f}\t'.format(x)+'{:10.0f}\t'.format(y)+'{:3.2f}\n'.format(swe))
    f.close()
    return len(uq_day)

In [23]:
def make_SMassim_file_both(STswe,STmeta,CSOdata,outFpath):

    f= open(outFpath,"w+")
    uq_day = np.unique(np.concatenate((STswe.index.date,CSOdata.dt.dt.date.values)))
    f.write('{:02.0f}\n'.format(len(uq_day)))

    stn = list(STswe.columns)

    for i in range(len(uq_day)):

        SThoy = STswe[STswe.index.date == uq_day[i]]
        CSOhoy = CSOdata[CSOdata.dt.dt.date.values == uq_day[i]]

        d=uq_day[i].day
        m=uq_day[i].month
        y=uq_day[i].year

        date = str(y)+' '+str(m)+' '+str(d)

        stn_count = len(stn) + len(CSOhoy)

        f.write(date+' \n')
        f.write(str(stn_count)+' \n')

        #go through snotel stations for that day 
        ids = 100
        if len(SThoy) > 0:
            for k in stn:
                ids = ids + 1 
                x = STmeta.easting.values[STmeta.code.values == k][0]
                y = STmeta.northing.values[STmeta.code.values == k][0]
                swe = SThoy[k].values[0]
                f.write('{:3.0f}\t'.format(ids)+'{:10.0f}\t'.format(x)+'{:10.0f}\t'.format(y)+'{:3.2f}\n'.format(swe))    
        #go through cso obs for that day 
        if len(CSOhoy) > 0:
            for c in range(len(CSOhoy)):
                ids = ids + 1 
                x= CSOhoy.x[CSOhoy.index[c]]
                y=CSOhoy.y[CSOhoy.index[c]]
                swe=CSOhoy.swe[CSOhoy.index[c]]
                f.write('{:3.0f}\t'.format(ids)+'{:10.0f}\t'.format(x)+'{:10.0f}\t'.format(y)+'{:3.2f}\n'.format(swe))
    f.close()
    return len(uq_day)

In [6]:
def SMassim_ensemble_both(STswe,STmeta,CSOdata,var,path):
    '''
    STmeta: this is the geodataframe containing all snotel stations
    STswe: this is a dataframe containing all snotel swe
    CSOdata: this is the geodataframe containing all CSO data    
    var: this is the landscape characteristic that will be made into an assimilation ensemble 
        'all': assimilate all inputs to SM
        'elev': assimilate each of n elevation bands. 
            Default = breaks elevation range into 5 bands
        'slope': assimilate each of n slope bands. 
            Default = breaks slope range into 5 bands
        'tc': assimilate each of n terrain complexity score bands. 
            Default = breaks tc score range into 5 bands
        'delta_day': sets a minimum number of days between assimilated observations. 
            -> only 1 observation is selected each day
        'M': assimilate data from each month
        'lc': assimilate data from each land cover class
        'aspect': assimilate data from each aspect N, E, S, W
    path: path to put all output SM .gdat files
    '''

    #create directory with initiation date for ensemble if it doesn't exist
    !mkdir -p $path
    outFpath = SMpath+'swe_assim/swe_obs_test.dat'
    if var == 'all':
        newST = STmeta
        newSTswe = STswe
        newCSO = CSOdata
        num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
        #edit .inc file
        replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
        #compile SM        
        %cd $codepath
        ! ./compile_snowmodel.script
        #run snowmodel 
        %cd $SMpath
        ! ./snowmodel
        #move swed.gdat file 
        oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
        nSWEpath = path + '/snotel_all_swed.gdat'
        !mv $oSWEpath $nSWEpath     
        
        
    elif var == 'elev':
        edges = np.histogram_bin_edges(CSOdata.dem_elev,bins=5, range=(CSOdata.dem_elev.min(),CSOdata.dem_elev.max()))
        print('edges:',edges)
        labs = np.arange(0,len(edges)-1,1)
        print('labels:',labs)
        bins = pd.cut(STmeta['dem_elev'], edges,labels=labs)
        STmeta['elev_bin']=bins 
        bins = pd.cut(CSOdata['dem_elev'], edges,labels=labs)
        CSOdata['elev_bin']=bins 
        for lab in labs:
            newST = STmeta[STmeta.elev_bin == lab]
            newCSO = CSOdata[CSOdata.elev_bin == lab]
            newSTswe = STswe[np.intersect1d(STswe.columns, newST.code.values)] 
            num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/snotel_elev_'+str(lab)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath                  
        
            
            
    elif var == 'slope':
        edges = np.histogram_bin_edges(CSOdata.slope,bins=5, range=(CSOdata.slope.min(),CSOdata.slope.max()))
        print('edges:',edges)
        labs = np.arange(0,len(edges)-1,1)
        print('labels:',labs)
        bins = pd.cut(STmeta['slope'], edges,labels=labs)
        STmeta['slope_bin']=bins 
        bins = pd.cut(CSOdata['slope'], edges,labels=labs)
        CSOdata['slope_bin']=bins   
        for lab in labs:
            newST = STmeta[STmeta.slope_bin == lab]
            newCSO = CSOdata[CSOdata.slope_bin == lab]
            newSTswe = STswe[np.intersect1d(STswe.columns, newST.code.values)] 
            num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/snotel_slope_'+str(lab)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath                                 

    elif var == 'tc':
        edges = np.histogram_bin_edges(CSOdata.tc,bins=5, range=(CSOdata.tc.min(),CSOdata.tc.max()))
        print('edges:',edges)
        labs = np.arange(0,len(edges)-1,1)
        print('labels:',labs)
        bins = pd.cut(STmeta['tc'], edges,labels=labs)
        STmeta['tc_bin']=bins 
        bins = pd.cut(CSOdata['tc'], edges,labels=labs)
        CSOdata['tc_bin']=bins 
        for lab in labs:
            newST = STmeta[STmeta.tc_bin == lab]
            newCSO = CSOdata[CSOdata.tc_bin == lab]
            newSTswe = STswe[np.intersect1d(STswe.columns, newST.code.values)] 
            num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/snotel_tc_'+str(lab)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath     
          
            
    elif var == 'delta_day':
        import datetime
        CSOdata = CSOdata.sort_values(by='dt',ascending=True)
        CSOdata = CSOdata.reset_index(drop=True)        
        newST = STmeta
        Delta = [3,5,7,10]
        for dels in Delta:
            idx = [0]
            st = CSOdata.dt[0]
            for i in range(1,len(CSOdata)-1):
                date = CSOdata.dt.iloc[i]
                gap = (date - st).days
                if gap<=dels:
                    continue 
                else:
                    idx.append(i)
                    st = date
            newCSO = CSOdata[CSOdata.index.isin(idx)]
            newSTswe = STswe.iloc[::dels,:]
            num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/snotel_day_delta'+str(dels)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath            
            
            
    elif var == 'M':
        newST = STmeta
        mo = [11,12,1,2,3,4,5]#np.unique(STswe.index.month)
        for m in mo:
            newSTswe = STswe[STswe.index.month == m]
            newCSO = CSOdata[CSOdata[var] == m]            
            num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/snotel_M_'+str(m)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath  

            
            
    else: #works for 'M', 'lc', 'aspect'
        uq = np.unique(np.concatenate((STmeta[var].values,CSOdata[var].values)))
        for lab in uq:
            newST = STmeta[STmeta[var] == lab]
            newCSO = CSOdata[CSOdata[var] == lab]
            newSTswe = STswe[np.intersect1d(STswe.columns, newST.code.values)] 
            num_obs = make_SMassim_file_both(newSTswe,newST,newCSO,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(num_obs+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/snotel_'+var+'_'+str(lab)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath     